### Setup a model

In [ ]:
from langchain_ollama import ChatOllama

ollama_model = ChatOllama(model="llama3.1")

ollama_model

### Register a tool

In [ ]:
from langchain.tools import tool
from typing import Union, List

@tool
def get_weather(location: Union[str, List[str]] = None, locations: Union[str, List[str]] = None) -> str:
    """Get the weather for a location or list of locations.
    
    Args:
        location: A single location or list of locations.
        locations: Alias for location, often used by models when multiple locations are requested.
    """
    # Handle argument alias
    final_locs = location if location is not None else locations
    
    if final_locs is None:
        return "Please provide a location."

    if isinstance(final_locs, list):
        return "\n".join([f"The weather in {loc} is sunny." for loc in final_locs])
    return f"The weather in {final_locs} is sunny."

model_with_tools = ollama_model.bind_tools([get_weather])


### Call tool in execution loop

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
# Step 1: Model generates tool calls
messages = [
    HumanMessage(content="What is the weather like in Noida, Bangalore and New York?")
]
ai_messages = model_with_tools.invoke(messages)
messages.append(ai_messages)

# Step 2: Execute tools and collect results

for tool_call in ai_messages.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)
    
# Step 3: Pass results back to model for final response
responses = model_with_tools.invoke(messages)
print(responses.content)




### Force use of any tool or specific tool

### Parallel tool calls

### Streaming tool calls